In [ ]:
# Importing Libraries
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
#loading rating dataset
ratings = pd.read_csv("ratings.csv")
print(ratings.head())

   userId  movieId  rating    timestamp
0       1       17     4.0  944249077.0
1       1       25     1.0  944250228.0
2       1       29     2.0  943230976.0
3       1       30     5.0  944249077.0
4       1       32     5.0  943228858.0


In [ ]:
# loading movie dataset
movies = pd.read_csv("movies.csv")
print(movies.head())

   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  


# **MLOPs**

# MLFlow

In [ ]:
import mlflow
import mlflow.sklearn
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise import accuracy
import pandas as pd


mlflow.set_tracking_uri(uri="http://127.0.0.1:5000")
mlflow.set_experiment("recommendation_system_svd")

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings[['user_id', 'movie_id', 'rating']], reader)

train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)

with mlflow.start_run():

    svd = SVD(n_factors=50, lr_all=0.005)

    mlflow.log_param('n_factors', 50)
    mlflow.log_param('lr_all', 0.005)

    svd.fit(train_data)

    predictions = svd.test(test_data)

    rmse = accuracy.rmse(predictions)
    mlflow.log_metric('RMSE', rmse)

mlflow.end_run()


MlflowException: API request to http://127.0.0.1:5000/api/2.0/mlflow/experiments/get-by-name failed with exception HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /api/2.0/mlflow/experiments/get-by-name?experiment_name=recommendation_system_svd (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7b01c2ffb160>: Failed to establish a new connection: [Errno 111] Connection refused'))

# Prompt Engineering

In [ ]:
model = pd.merge(ratings, movies, on='movieId')

In [ ]:
# recommend movies function based on rating
def recommend_movies_rating(rating_input):


    # filter data to get movies that user ask for
    recommended_movies = model[model['rating'] == rating_input]

    if recommended_movies.empty:
        return ["Sorry, no movies found with that rating."]

    # extract the movie titles from the filtered DataFrame
    return recommended_movies['title'].unique()[:5]

# chatbot function
def movies_rating_chatbot():
    print("Welcome to the Movie Recommendation Bot!")

    while True:
        # ask the user for a rating to search for movies
        user_input = input("Enter a rating to find movies or 'quit' to exit): ").strip()

        if user_input.lower() == 'quit':
            print("Goodbye! Enjoy your movies!")
            break

        try:
            # convert input to float for rating comparison
            rating_input = float(user_input)

            if rating_input < 1 or rating_input > 5:
                print("Please enter a rating between 1 and 5.")
                continue

        except ValueError:
            print("Please enter a valid number.")
            continue

        # get recommendations based on rating
        recommendations = recommend_movies_rating(rating_input)

        # run recommended movies
        if len(recommendations) > 0:
            print(f"Movies with a rating of {rating_input}: {', '.join(recommendations)}")
        else:
            print(f"Sorry, no movies found with a rating of {rating_input}.")

# run chatbot
movies_rating_chatbot()


Welcome to the Movie Recommendation Bot!
Movies with a rating of 2.0: City of Lost Children, The (Cité des enfants perdus, La) (1995), Babe (1995), Forrest Gump (1994), Fargo (1996), Casablanca (1942)
Please enter a rating between 1 and 5.
Movies with a rating of 4.0: Sense and Sensibility (1995), Living in Oblivion (1995), Queen Margot (Reine Margot, La) (1994), Swimming with Sharks (1995), Nelly & Monsieur Arnaud (1995)
Goodbye! Enjoy your movies!
